In [1]:
import numpy as np
from scipy.integrate import quad

## Definite Integrals (Numerical)

In [4]:
f = lambda x: 1/np.sqrt(x)
quad(f, 0, 1)

(1.9999999999999984, 5.773159728050814e-15)

## Lorentzian integral over the range [0, ∞)

In [8]:
a = 1.0
f = lambda x: 1/(x*x+a*a)**2
quad(f, 0, np.inf)

(0.7853981633974483, 1.944861002212378e-09)

## Slow Converging Integrals (throws warning)

In [11]:
f = lambda x: np.sin(x)**2/x**2
quad(f, 0, np.inf)

C:\Users\Admin\AppData\Local\Temp\ipykernel_21848\3473048144.py:2: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad(f, 0, np.inf)


(1.5708678849453777, 0.0015587759422623915)

## Expansion Coefficients (numerical)

In [17]:
psi = lambda x: x*(a-x)
psi2 = lambda x: psi(x)**2
psin = lambda n, x: np.sqrt(2/a)*np.sin(n*pi*x/a)
f = lambda x: psi(x)*psin(n,x)
pi = np.pi
a = 1
A = quad(psi2, 0, a)[0] # norm
for n in range(1,5):
    print(n, quad(f, 0, a)[0]/np.sqrt(A))

1 0.9992772459953339
2 5.870202015831392e-17
3 0.03701026837019757
4 -6.221452546292864e-17


## Differential Equation Solver

In [24]:
from scipy.integrate import odeint
def drag(v, t):
    b = 1.0
    return -b*v

t = np.linspace(0,1,5) # time grid
v = 1.0 # initial velocity
odeint(drag, v, t)

array([[1.        ],
       [0.7788008 ],
       [0.60653067],
       [0.47236657],
       [0.36787947]])

## Coupled ODE

In [27]:
def drag2(y, t):
    b = 1.0
    v = y[1]
    return [v, -b*v]
y = [0., 1.] # y=[x, v], initial pos and velocity
odeint(drag2, y, t)

array([[0.        , 1.        ],
       [0.22119922, 0.77880078],
       [0.39346933, 0.60653067],
       [0.52763344, 0.47236656],
       [0.63212055, 0.36787945]])

## Special Functions

In [30]:
from scipy.special import erf, gamma, lambertw, spherical_jn
x = 0.5
erf(x), gamma(x), lambertw(x)

(0.5204998778130465, 1.7724538509055159, (0.35173371124919584+0j))

## Solving Equations

In [33]:
from scipy.optimize import fsolve, bisect
f = lambda x: (x-3)*np.exp(x)+3 # freq, x = hν/kT
fsolve(f, [1., 3.])

array([2.27881059e-12, 2.82143937e+00])

In [35]:
 lambertw(-3/np.e**3)+3

(2.8214393721220787+0j)

In [37]:
bisect(f, 1., 5.)

2.821439372122768

## Finding Bessel Function Zeroes

In [40]:
f = lambda x: spherical_jn(n,x)
n = 1
bisect(f, 1., 5.), bisect(f, 5., 10.)

(4.493409457909365, 7.725251836938014)

## System of Equations

In [43]:
from scipy.linalg import solve, eigh
A = np.random.random(9).reshape(3,3)
A = A + np.transpose(A) # form a symmetric matrix
A

array([[1.25669209, 1.01882319, 0.68061303],
       [1.01882319, 1.32513213, 0.45295705],
       [0.68061303, 0.45295705, 1.76160913]])

In [45]:
c = np.random.random(3)
c

array([0.18444752, 0.03472796, 0.15162535])

## Solving System of Equations

In [50]:
x = solve(A, c)
x

array([ 0.31841547, -0.22582285,  0.0211146 ])

## Verifying Solution Through Substitution

In [53]:
 np.dot(A, x)

array([0.18444752, 0.03472796, 0.15162535])

## Eigenvalues and Eigenvectors

In [56]:
vals, vecs = eigh(A)
vals, vecs

(array([0.24629595, 1.21268666, 2.88445074]),
 array([[ 0.74882693, -0.29797562, -0.59200402],
        [-0.64717995, -0.52132342, -0.55621939],
        [-0.14288574,  0.79964519, -0.58322486]]))

## 1st Eigenvector Normalized

In [59]:
np.sum(vecs[:,0]**2)

0.9999999999999998

## Orthogonality of Eigenvectors

In [65]:
np.dot(vecs[:,0], vecs[:,1]) # Should resultin a very small value (zero)

-4.163336342344337e-17

## Numba (JIT)

In [77]:
import numpy as np, time
from numba import jit

def calcNorm(u0, u1):
    u2 = - u0
    for n in range(1, N-1):
        u2[n] += u1[n-1] + u1[n+1] # left, right
    return u2
N = 5000
u0 = np.random.random(N)
u1 = 0.9*u0

tc = time.perf_counter()
for i in range(2000):
    u2 = calcNorm(u0, u1)
    u0, u1 = u1, u2

tc = time.perf_counter() - tc
print(f"Without Numba: {tc}")

@jit
def calc(u0, u1):
    u2 = - u0
    for n in range(1, N-1):
        u2[n] += u1[n-1] + u1[n+1] # left, right
    return u2

N = 5000
u0 = np.random.random(N)
u1 = 0.9*u0

tc = time.perf_counter()
for i in range(2000):
    u2 = calc(u0, u1)
    u0, u1 = u1, u2

tc = time.perf_counter() - tc
print(f"With Numba: {tc}")


Without Numba: 9.29974809999112
With Numba: 0.26912110002012923
